In [2]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

In [3]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


# Exclusive

In [4]:
expVar: list[str] = ["process", "iteration", "size"]
resVar: str = "Exclusive"

trainDF: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

testDF: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    list_csvDir=list_csvDir,
    resVar=resVar,
)

testDF = testDF.reset_index()
trainDF = trainDF.reset_index()

testDF["functionName"] = testDF["Name"]
trainDF["functionName"] = trainDF["Name"]

functionNames: list[str] = sorted(list(set(trainDF["Name"])))

dict_symbols = {}
for elem in expVar:
    dict_symbols[elem] = symbols(elem, real=True)

target_env = [
    (dict_symbols["size"], test_lulesh_sizes[-1]),
    (dict_symbols["iteration"], test_lulesh_iterations[-1]),
    (dict_symbols["process"], test_lulesh_processes[-1]),
]

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

In [36]:
dict_models: dict[str, dict[str, any]] = {}
for functionName in functionNames:

    trainDF_perFunc: pd.DataFrame = trainDF[trainDF["Name"] == functionName]
    testDF_perFunc: pd.DataFrame = testDF[testDF["Name"] == functionName]

    # 総実行時間

    resVar = "Exclusive"
    # Extra-P への入力ファイル作成
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=testDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)
    # Extra-P の実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout

    res_str_tmp: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep Adjusted",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout
    print(f"{functionName}\nres_str_tmp = {res_str_tmp}")

    # 取得した Extra-P の出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)
    # 総実行時間の予測
    model_sympy_all = sympify(res_str, locals=dict_symbols)

    # 1コール当たりの実行時間

    resVar = "ExclusivePerCall"
    # Extra-P への入力ファイル作成
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=testDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)
    # Extra-P の実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout

    # 取得したExtra-Pの出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)
    # 総実行時間の予測
    model_sympy_perCall = sympify(res_str, locals=dict_symbols)

    # 関数コール回数の予測

    # 関数コール回数予測のためのモデルを構築
    resVar = "#Call"
    bestModelDict: dict = return_bestModelObject(
        inputDF=trainDF_perFunc,
        list_expVar=expVar,
        list_resVar=[resVar],
        list_modelName=list_modelName,
    )
    bestModel = bestModelDict["object"]

    dict_models_perFunc: dict[str, any] = {
        "time_all": model_sympy_all,
        "time_perFunc": model_sympy_perCall,
        "call_perFunc": bestModel,
    }
    dict_models[functionName] = dict_models_perFunc

Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]


.TAU_application
res_str_tmp = 		Adjusted R^2: 1.47E-01



Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters co

MPI_Allreduce()
res_str_tmp = 		Adjusted R^2: 9.93E-01



Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parame

...

Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters co

void_ParseCommandLineOptions(int_char_**_Int_t_cmdLineOpts
res_str_tmp = 		Adjusted R^2: -3.83E-02



Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters co

void_VerifyAndWriteFinalOutput(Real_t_Domain
res_str_tmp = 		Adjusted R^2: 9.67E-01



Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]


In [6]:
print(dict_models_perFunc)
print(f'time_all = {type(dict_models_perFunc["time_all"])}')
print(f'time_perFunc = {type(dict_models_perFunc["time_perFunc"])}')
print(f'call_perFunc = {type(dict_models_perFunc["call_perFunc"])}')

{'time_all': -3.3193430488293343e-13*size**(5/4)*log(iteration)*log(process)/log(2)**2 + 1.7811331690654008e-10*size**(5/4) - 1.1655929556898692e-7*log(process)/log(2) + 1.226624900744705e-6, 'time_perFunc': 3.260324255014677e-16*iteration**3*log(iteration)/log(2) + 2.328094155724892e-12*size**3*log(size)/log(2) + 9.516362036684537e-5, 'call_perFunc': <lib.lab_lib.ModelMultipleEquationForMultipleRegression object at 0x7f1f10bdce50>}
time_all = <class 'sympy.core.add.Add'>
time_perFunc = <class 'sympy.core.add.Add'>
call_perFunc = <class 'lib.lab_lib.ModelMultipleEquationForMultipleRegression'>


In [7]:
# 集計をする
# それぞれの予測対象ケースで総実行時間の相対誤差率を算出する
# 算出された相対誤差率をプロットする

resVar: str = "Exclusive"
dict_resultDF_perFunc = {}
for elem_process in test_lulesh_processes:
    for elem_iteration in test_lulesh_iterations:
        for elem_size in test_lulesh_sizes:
            
            print(f"process={elem_process}, iteration={elem_iteration}, size={elem_size}")
            
            testDF_oneCase: pd.DataFrame = ret_averaged_rawDF_lulesh(
                list_process=[elem_process],
                list_iteration=[elem_iteration],
                list_size=[elem_size],
                list_csvDir=list_csvDir,
                resVar=resVar,
            )
            testDF_oneCase["functionName"] = testDF_oneCase["Name"]

            target_env = [
                (dict_symbols["size"], elem_size),
                (dict_symbols["iteration"], elem_iteration),
                (dict_symbols["process"], elem_process),
            ]

            functionNames: list[str] = sorted(list(testDF_oneCase["Name"]))
            list_series: list[pd.Series] = []
            for functionName in functionNames:
                testDF_oneCase_perFunc: pd.DataFrame = testDF_oneCase[
                    testDF_oneCase["Name"] == functionName
                ]

                if len(testDF_oneCase_perFunc) != 1:
                    warnings.warn("len(testDF_oneCase_perFunc) != 1")
                    exit

                # 総実行時間の予測
                Exclusive_predicted_all: float = (
                    dict_models[functionName]["time_all"].subs(target_env).evalf()
                )
                # 1コール当たりの実行時間
                Exclusive_predicted_perFunc: float = (
                    dict_models[functionName]["time_perFunc"].subs(target_env).evalf()
                )
                # コール回数の予測
                predicted_call: float = float(
                    np.array(
                        dict_models[functionName]["call_perFunc"].predict(
                            inputDF=testDF_oneCase_perFunc[expVar]
                        )
                    )
                )

                # 実測値の取得
                Exclusive_real_time: float = testDF_oneCase_perFunc.reset_index().loc[
                    0
                ][resVar]
                real_call: float = testDF_oneCase_perFunc.reset_index().loc[0]["#Call"]

                _series: pd.Series = pd.Series(
                    {
                        "functionName": functionName,
                        f"{resVar}_real_time": Exclusive_real_time,
                        f"{resVar}_predicted_all": Exclusive_predicted_all,
                        f"{resVar}_predicted_from_perCall": Exclusive_predicted_perFunc
                        * predicted_call,
                    }
                )
                list_series.append(_series)

            resultDF_perFunc: pd.DataFrame = pd.concat(list_series, axis=1).T
            resultDF_perFunc = add_relativeErrorRateCol(
                inputDF=resultDF_perFunc,
                real_colName=f"{resVar}_real_time",
                predicted_colName=f"{resVar}_predicted_all",
                targetColName=f"相対誤差率（Extra-P）",
            )
            resultDF_perFunc = add_relativeErrorRateCol(
                inputDF=resultDF_perFunc,
                real_colName=f"{resVar}_real_time",
                predicted_colName=f"{resVar}_predicted_from_perCall",
                targetColName=f"相対誤差率（組合せ）",
            )
            # print(resultDF_perFunc)
            dict_resultDF_perFunc[f"p{elem_process}i{elem_iteration}s{elem_size}"] = resultDF_perFunc

process=512, iteration=256, size=64
process=512, iteration=256, size=96
process=512, iteration=256, size=128
process=512, iteration=512, size=64
process=512, iteration=512, size=96
process=512, iteration=512, size=128
process=512, iteration=1024, size=64
process=512, iteration=1024, size=96
process=512, iteration=1024, size=128
process=729, iteration=256, size=64
process=729, iteration=256, size=96
process=729, iteration=256, size=128
process=729, iteration=512, size=64
process=729, iteration=512, size=96
process=729, iteration=512, size=128
process=729, iteration=1024, size=64
process=729, iteration=1024, size=96
process=729, iteration=1024, size=128
process=1000, iteration=256, size=64
process=1000, iteration=256, size=96
process=1000, iteration=256, size=128
process=1000, iteration=512, size=64
process=1000, iteration=512, size=96
process=1000, iteration=512, size=128
process=1000, iteration=1024, size=64
process=1000, iteration=1024, size=96
process=1000, iteration=1024, size=128


In [8]:
dict_resultDF_perFunc.keys()

dict_keys(['p512i256s64', 'p512i256s96', 'p512i256s128', 'p512i512s64', 'p512i512s96', 'p512i512s128', 'p512i1024s64', 'p512i1024s96', 'p512i1024s128', 'p729i256s64', 'p729i256s96', 'p729i256s128', 'p729i512s64', 'p729i512s96', 'p729i512s128', 'p729i1024s64', 'p729i1024s96', 'p729i1024s128', 'p1000i256s64', 'p1000i256s96', 'p1000i256s128', 'p1000i512s64', 'p1000i512s96', 'p1000i512s128', 'p1000i1024s64', 'p1000i1024s96', 'p1000i1024s128'])

In [9]:
len(dict_resultDF_perFunc.keys())

27

In [10]:
_all :list [float] = []
_perCall :list[float] = []
_condition :list[str] = []
for key in dict_resultDF_perFunc.keys():
    targetDF = dict_resultDF_perFunc[key]
    
    relativeErrorOnTarget_perCall :float = ret_relativeError_fromSumOfCol(
        inputDF = targetDF,
        realCol = "Exclusive_real_time",
        predictedCol = "Exclusive_predicted_from_perCall",
    )
    relativeErrorOnTarget_all :float = ret_relativeError_fromSumOfCol(
        inputDF = targetDF,
        realCol = "Exclusive_real_time",
        predictedCol = "Exclusive_predicted_all",
    )
    
    _condition.append(key)
    _all.append(relativeErrorOnTarget_all)
    _perCall.append(relativeErrorOnTarget_perCall)
    
resultDF :pd.DataFrame = pd.DataFrame(data={"Extra-Pのみ":_all, "組合せ":_perCall, "条件":_condition})
resultDF 

,Extra-Pのみ,組合せ,条件
0,12.6333141263911,0.507208978117047,p512i256s64
1,8.78762731489746,1.68957291981639,p512i256s96
2,9.21255948992760,0.0990770132824140,p512i256s128
3,13.3301888200060,0.160422824240916,p512i512s64
4,9.53668899316149,0.170231071504648,p512i512s96
5,8.87409086042504,0.425618036798869,p512i512s128
6,15.7641189708340,2.82949404518662,p512i1024s64
7,8.54672858981889,0.760886080019178,p512i1024s96
8,8.40751501760401,0.417010936391316,p512i1024s128
9,14.3130133544459,3.76154927956209,p729i256s64


In [11]:
print(resultDF.to_csv())

,Extra-Pのみ,組合せ,条件
0,12.6333141263911,0.507208978117047,p512i256s64
1,8.78762731489746,1.68957291981639,p512i256s96
2,9.21255948992760,0.0990770132824140,p512i256s128
3,13.3301888200060,0.160422824240916,p512i512s64
4,9.53668899316149,0.170231071504648,p512i512s96
5,8.87409086042504,0.425618036798869,p512i512s128
6,15.7641189708340,2.82949404518662,p512i1024s64
7,8.54672858981889,0.760886080019178,p512i1024s96
8,8.40751501760401,0.417010936391316,p512i1024s128
9,14.3130133544459,3.76154927956209,p729i256s64
10,7.93703415766320,2.02957338955345,p729i256s96
11,8.46494699503024,0.684945221786042,p729i256s128
12,14.6495731810092,0.957466818284560,p729i512s64
13,9.26813284540945,0.656547350916434,p729i512s96
14,8.25169349468960,0.340345895005132,p729i512s128
15,14.8793930637566,0.0817976553047962,p729i1024s64
16,9.52582738419136,0.266803086211699,p729i1024s96
17,8.65520760392545,0.440815082810937,p729i1024s128
18,0.406657331497498,4.00981476929486,p1000i256s64
19,7.27377910767353,0.638296016

In [12]:
import plotly.graph_objects as go

barData_all = go.Bar(x = resultDF["条件"], y = resultDF["Extra-Pのみ"])
barData_perCall = go.Bar(x = resultDF["条件"], y = resultDF["組合せ"])


# 総実行時間から総実行時間を予測する

In [29]:


# 1. DFの作成
resVar :str = "Exclusive"
list_series_toMakeDF :list[pd.Series] = []
for elem_process in train_lulesh_processes:
    for elem_iteration in train_lulesh_iterations:
        for elem_size in train_lulesh_sizes:
            trainDF_oneCase :pd.DataFrame = ret_averaged_rawDF_lulesh(
                list_process=[elem_process],
                list_iteration=[elem_iteration],
                list_size=[elem_size],
                list_csvDir=list_csvDir,
                resVar=resVar
            )
            execTime_inThisCase :float = sum(trainDF_oneCase[resVar])
            _series :pd.Series = pd.Series({
                "process":elem_process,
                "iteration":elem_iteration,
                "size": elem_size,
                resVar: execTime_inThisCase,
            })
            list_series_toMakeDF.append(_series)
inputDFtoExtraP :pd.DataFrame = pd.DataFrame(list_series_toMakeDF)


# 2. Extra-Pへの入力ファイル作成
str_ExtraPinputData :str = gen_ExtraPinputDataFromDF(
    inputDF = inputDFtoExtraP,
    expVar = expVar,
    resVar = resVar
)
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)
# 3. Extra-Pの実行
res_str :str = subprocess.run(
    "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep Model",
    stdout=subprocess.PIPE,
    text=True,
    shell=True,
).stdout
# 4. 受け取った出力の整形
res_str = res_str.replace("Model: ", "")
res_str = convert_log(res_str)
# 5. モデル化
model_sympy_whole = sympify(res_str, locals=dict_symbols)



Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: |          | [00:00<?]/usr/local/lib/python3.10/site-packages/extrap/modelers/single_parameter/basic.py:273: UserWarning: Number of measurements for a parameter needs to be at least 5 in order to create a performance model.
  warnings.warn(
Generating models: 100%|██████████| [00:00<00:00]


In [21]:
list_series :list[pd.Series]= []
for elem_process in test_lulesh_processes:
    for elem_iteration in test_lulesh_iterations:
        for elem_size in test_lulesh_sizes:
            
            print(f"process={elem_process}, iteration={elem_iteration}, size={elem_size}")
            condition :str = f"p{elem_process}i{elem_iteration}s{elem_size}"
            
            # 実時間の取得
            testDF_oneCase: pd.DataFrame = ret_averaged_rawDF_lulesh(
                list_process=[elem_process],
                list_iteration=[elem_iteration],
                list_size=[elem_size],
                list_csvDir=list_csvDir,
                resVar=resVar,
            )
            exclusive_real_time :float = sum(testDF_oneCase[resVar])

            target_env = [
                (dict_symbols["size"], elem_size),
                (dict_symbols["iteration"], elem_iteration),
                (dict_symbols["process"], elem_process),
            ]
            # 総実行時間の予測
            exclusive_predicted_time: float = (
                model_sympy_whole.subs(target_env).evalf()
            )

            _series :pd.Series = pd.Series({
                "exclusive_real_time" : exclusive_real_time,
                "exclusive_predicted_time": exclusive_predicted_time,
                "条件" :condition
            })

            list_series.append(_series)

resultDF2 :pd.DataFrame = pd.DataFrame(list_series)
resultDF2 = add_relativeErrorRateCol(inputDF=resultDF2, real_colName="exclusive_real_time", predicted_colName="exclusive_predicted_time", targetColName="相対誤差率（all to all）")

resultDF2


process=512, iteration=256, size=64
process=512, iteration=256, size=96
process=512, iteration=256, size=128
process=512, iteration=512, size=64
process=512, iteration=512, size=96
process=512, iteration=512, size=128
process=512, iteration=1024, size=64
process=512, iteration=1024, size=96
process=512, iteration=1024, size=128
process=729, iteration=256, size=64
process=729, iteration=256, size=96
process=729, iteration=256, size=128
process=729, iteration=512, size=64
process=729, iteration=512, size=96
process=729, iteration=512, size=128
process=729, iteration=1024, size=64
process=729, iteration=1024, size=96
process=729, iteration=1024, size=128
process=1000, iteration=256, size=64
process=1000, iteration=256, size=96
process=1000, iteration=256, size=128
process=1000, iteration=512, size=64
process=1000, iteration=512, size=96
process=1000, iteration=512, size=128
process=1000, iteration=1024, size=64
process=1000, iteration=1024, size=96
process=1000, iteration=1024, size=128


,exclusive_real_time,exclusive_predicted_time,条件,相対誤差率（all to all）
0,132.149312,82.5581576693674,p512i256s64,37.5266080780388
1,471.983715,274.231705131693,p512i256s96,41.8980578293707
2,1143.805716,636.258741337536,p512i256s128,44.3735302064036
3,263.338648,165.826470359938,p512i512s64,37.0291935881498
4,955.757055,550.615148688673,p512i512s96,42.3896327974435
5,2296.174716,1277.39203900681,p512i512s128,44.3686915433420
6,536.093316,332.363095741079,p512i1024s64,38.0027532959863
7,1890.687721,1103.38203580263,p512i1024s96,41.6412333065201
8,4578.817049,2559.65863434536,p512i1024s128,44.0978181335054
9,143.068651,81.9904840049816,p729i256s64,42.6915098023799


In [28]:
mergedDF :pd.DataFrame = pd.merge(left=resultDF, right=resultDF2.drop(columns=["exclusive_real_time", "exclusive_predicted_time"]), on="条件")
print(mergedDF.to_csv())

,Extra-Pのみ,組合せ,条件,相対誤差率（all to all）
0,12.6333141263911,0.507208978117047,p512i256s64,37.5266080780388
1,8.78762731489746,1.68957291981639,p512i256s96,41.8980578293707
2,9.21255948992760,0.0990770132824140,p512i256s128,44.3735302064036
3,13.3301888200060,0.160422824240916,p512i512s64,37.0291935881498
4,9.53668899316149,0.170231071504648,p512i512s96,42.3896327974435
5,8.87409086042504,0.425618036798869,p512i512s128,44.3686915433420
6,15.7641189708340,2.82949404518662,p512i1024s64,38.0027532959863
7,8.54672858981889,0.760886080019178,p512i1024s96,41.6412333065201
8,8.40751501760401,0.417010936391316,p512i1024s128,44.0978181335054
9,14.3130133544459,3.76154927956209,p729i256s64,42.6915098023799
10,7.93703415766320,2.02957338955345,p729i256s96,42.4680392159481
11,8.46494699503024,0.684945221786042,p729i256s128,44.3794929769896
12,14.6495731810092,0.957466818284560,p729i512s64,40.1699403867848
13,9.26813284540945,0.656547350916434,p729i512s96,42.8138711596887
14,8.25169349468960,0.3403458950

In [31]:
# 実時間の取得
trainDF_oneCase: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=[train_lulesh_processes[-1]],
    list_iteration=[train_lulesh_iterations[-1]],
    list_size=[train_lulesh_sizes[-1]],
    list_csvDir=list_csvDir,
    resVar=resVar,
)
exclusive_real_time :float = sum(trainDF_oneCase[resVar])

target_env = [
    (dict_symbols["size"], train_lulesh_sizes[-1]),
    (dict_symbols["iteration"], train_lulesh_iterations[-1]),
    (dict_symbols["process"], train_lulesh_processes[-1]),
]
# 総実行時間の予測
exclusive_predicted_time: float = (
    model_sympy_whole.subs(target_env).evalf()
)

_series :pd.Series = pd.Series({
    "exclusive_real_time" : exclusive_real_time,
    "exclusive_predicted_time": exclusive_predicted_time,
    "条件" :condition
})

_series

exclusive_real_time                26.232715
exclusive_predicted_time    17.8654504325857
条件                            p1000i1024s128
dtype: object

In [32]:
print(train_lulesh_processes[-1])
print(train_lulesh_iterations[-1])
print(train_lulesh_sizes[-1])

343
128
48


In [33]:
_process = 64
_iteration = 64
_size = 24

# 実時間の取得
trainDF_oneCase: pd.DataFrame = ret_averaged_rawDF_lulesh(
    list_process=[_process],
    list_iteration=[_iteration],
    list_size=[_size],
    list_csvDir=list_csvDir,
    resVar=resVar,
)
exclusive_real_time :float = sum(trainDF_oneCase[resVar])

target_env = [
    (dict_symbols["size"], _size),
    (dict_symbols["iteration"], _iteration),
    (dict_symbols["process"], _process),
]
# 総実行時間の予測
exclusive_predicted_time: float = (
    model_sympy_whole.subs(target_env).evalf()
)

_series :pd.Series = pd.Series({
    "exclusive_real_time" : exclusive_real_time,
    "exclusive_predicted_time": exclusive_predicted_time,
    # "条件" :condition
})

_series

exclusive_real_time                 3.383667
exclusive_predicted_time    4.26771526403966
dtype: object